## Inicialização do Navegador

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

## Definição das funções de busca do Google Shopping, Buscape e das funções de verificação dos termos banidos e verificação dos nomes dos produtos

In [ ]:
def verificar_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for termo in lista_termos_banidos:
        if termo in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_nomes_produtos(lista_nomes_produto, nome):
    tem_nomes_produto = True
    for termo in lista_nomes_produto:
        if termo not in nome:
            tem_nomes_produto = False
    return tem_nomes_produto

def busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo):

    produto.lower()
    termos_banidos.lower()

    lista_nomes_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")

    lista_ofertas_encontradas = []

    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    driver.get("https://www.google.com.br/")

    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto, Keys.ENTER)

    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')

    for elemento in elementos:
        if "Shopping" in elemento.text:
            elemento.click()
            break

    lista_resultados = driver.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text.lower()
 
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)
        tem_nomes_produto = verificar_nomes_produtos(lista_nomes_produto, nome)
    
        if tem_termos_banidos == False and tem_nomes_produto:
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            if preco_minimo <= preco <= preco_maximo:

                child = resultado.find_element(By.CLASS_NAME, 'XrAfOe')
                parent = child.find_element(By.XPATH, '..')
                link = parent.get_attribute('href')

                lista_ofertas_encontradas.append((nome, preco, link))
    return lista_ofertas_encontradas



def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    produto.lower()
    termos_banidos.lower()

    lista_nomes_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")

    lista_ofertas_encontradas = []   
    
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    driver.get('https://www.buscape.com.br/')

    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    while len(driver.find_elements(By.CLASS_NAME, 'Select_Select__1S7HV')) < 1:
        time.sleep(1)

    lista_resultados = driver.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text

        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
    
        link = resultado.get_attribute('href')

        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)
        tem_nomes_produto = verificar_nomes_produtos(lista_nomes_produto, nome)

        if tem_termos_banidos == False and tem_nomes_produto:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas_encontradas.append((nome, preco, link))
    return lista_ofertas_encontradas


## Criação da tabela das ofertas encontradas

In [ ]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]

    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns = ['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None

    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns = ['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

display(tabela_ofertas)
    

## Exportação para o Excel

In [ ]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

## Envio da tabela por e-mail

In [ ]:
import win32com.client

if len(tabela_ofertas) > 0:
    outlook = win32com.client.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'vincenzofortests@gmail.com'
    mail.Subject = 'Produto(s) encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Esses são os produtos em oferta que estão dentro da faixa de preço desejada</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att.</p>
    """
    mail.Send()